In [15]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plotting_base import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.1f}".format})
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash
from dash_bootstrap_templates import load_figure_template
from jupyter_dash import JupyterDash
from IPython import get_ipython
import dash_bootstrap_components as dbc

load_figure_template("darkly")
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
try:
    shell = str(get_ipython())
    if "ZMQInteractiveShell" in shell:
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    elif shell == "TerminalInteractiveShell":
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    else:
        app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
except NameError:
    app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])

bg_color = "#0b0b18"

mufex_main = Mufex(use_test_net=False)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
candles = dl_ex_candles(exchange="mufex", symbol="BTCUSDT", timeframe="5m", candles_to_dl=400)
open = candles[:, 1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4], 1)

In [20]:
close

array([36575.7, 36550.9, 36579.5, 36622.2, 36666.5, 36636.8, 36628.9,
       36615.4, 36590.0, 36594.0, 36646.9, 36627.3, 36687.1, 36678.3,
       36680.9, 36607.4, 36671.9, 36710.5, 36669.0, 36719.9, 36758.2,
       36803.8, 36797.4, 36744.9, 36749.8, 36855.8, 36776.2, 36768.7,
       36811.3, 36826.0, 36878.0, 36774.0, 36763.6, 36777.0, 36807.6,
       36840.3, 36788.3, 36765.2, 36729.2, 36708.8, 36682.6, 36691.3,
       36736.4, 36719.9, 36683.7, 36708.6, 36760.1, 36763.0, 36802.6,
       36815.3, 36787.4, 36832.9, 36848.5, 36963.1, 36946.4, 36945.2,
       36884.6, 36883.4, 36948.2, 36886.6, 36937.8, 36891.2, 36922.5,
       36990.5, 36895.5, 36951.6, 36953.3, 36900.9, 36839.0, 36832.6,
       36922.1, 37077.7, 37070.7, 37180.8, 37233.2, 37323.7, 37640.1,
       37616.1, 37656.8, 37574.9, 37636.9, 37650.4, 37719.6, 37797.5,
       37951.3, 37926.1, 37800.2, 37931.9, 37758.7, 37651.5, 37639.1,
       37369.7, 37480.8, 37547.7, 37444.9, 37342.9, 37384.0, 37142.2,
       37139.4, 3694

In [21]:
rma = rma_tv_calc(source=close, length=14)
rma

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       36621.4, 36625.6, 36624.3, 36627.7, 36633.6, 36636.2, 36642.2,
       36650.4, 36661.4, 36671.1, 36676.4, 36681.6, 36694.1, 36699.9,
       36704.8, 36712.4, 36720.6, 36731.8, 36734.8, 36736.9, 36739.7,
       36744.6, 36751.4, 36754.1, 36754.9, 36753.0, 36749.9, 36745.1,
       36741.2, 36740.9, 36739.4, 36735.4, 36733.5, 36735.4, 36737.4,
       36742.0, 36747.3, 36750.1, 36756.0, 36762.6, 36777.0, 36789.1,
       36800.2, 36806.2, 36811.8, 36821.5, 36826.1, 36834.1, 36838.2,
       36844.2, 36854.7, 36857.6, 36864.3, 36870.7, 36872.8, 36870.4,
       36867.7, 36871.6, 36886.3, 36899.5, 36919.6, 36942.0, 36969.2,
       37017.2, 37059.9, 37102.6, 37136.3, 37172.1, 37206.2, 37242.9,
       37282.5, 37330.3, 37372.8, 37403.4, 37441.1, 37463.8, 37477.2,
       37488.8, 37480.3, 37480.3, 37485.1, 37482.3, 37472.3, 37466.0,
       37442.9, 37421.2, 37387.0, 37332.4, 37282.3, 37234.5, 37188.6,
       37140.4, 37

In [26]:
def plot_tv_rma(candles: np.array, indicator: np.array, ind_name: str = "RMA", ind_color: str = "#3EA3FF"):
    datetimes = pd.to_datetime(candles[:, 0], unit="ms")
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=datetimes,
                open=candles[:, 1],
                high=candles[:, 2],
                low=candles[:, 3],
                close=candles[:, 4],
                name="Candles",
            ),
            go.Scatter(
                x=datetimes,
                y=rma,
                name=ind_name,
                line_color=ind_color,
            ),
        ]
    )
    fig.update_layout(height=800, xaxis_rangeslider_visible=False)
    fig.show()

In [28]:
plot_tv_rma(candles=candles,indicator=rma, ind_name="TV RMA", ind_color="#E9FF00")